In [44]:
from pprint import pprint
import regex as re

# for future reference
import sentencepiece as spm
import tiktoken

In [45]:
# we can use unicode ofc, but it has 2 problems:
# 1. it's 'live' standard, changing quite often
# 2. it has too many characters (100k+)

In [46]:
list("lorem ipsum dolor sit amet".encode("utf-8"))
# list("lorem ipsum dolor sit amet".encode("utf-16"))
# list("lorem ipsum dolor sit amet".encode("utf-32"))

# let's stick with utf-8 (it's denser in a way of not havingg too many 0s),
# but ...
# it has only 256 symbols

[108,
 111,
 114,
 101,
 109,
 32,
 105,
 112,
 115,
 117,
 109,
 32,
 100,
 111,
 108,
 111,
 114,
 32,
 115,
 105,
 116,
 32,
 97,
 109,
 101,
 116]

## BPE

In [47]:
# find the most frequent pair -> replace it with a new symbol (extending vocabulary) -> repeat

In [48]:
text = """
A Programmer’s Introduction to Unicode 
March 3, 2017 · Coding · 25 Comments

Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.

A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.

I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.
"""

In [49]:
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))
print(tokens)

[10, 65, 32, 80, 114, 111, 103, 114, 97, 109, 109, 101, 114, 226, 128, 153, 115, 32, 73, 110, 116, 114, 111, 100, 117, 99, 116, 105, 111, 110, 32, 116, 111, 32, 85, 110, 105, 99, 111, 100, 101, 32, 10, 77, 97, 114, 99, 104, 32, 51, 44, 32, 50, 48, 49, 55, 32, 194, 183, 32, 67, 111, 100, 105, 110, 103, 32, 194, 183, 32, 50, 53, 32, 67, 111, 109, 109, 101, 110, 116, 115, 10, 10, 239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32, 84, 104, 101, 32, 118, 101, 114, 121, 32, 110, 97, 109, 101, 32, 115, 116, 114, 105, 1

In [50]:
def get_stats(tokens):
    counts = {}
    for pair in zip(tokens, tokens[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

In [51]:
stats = get_stats(tokens)
print(stats)

{(10, 65): 2, (65, 32): 2, (32, 80): 1, (80, 114): 1, (114, 111): 7, (111, 103): 4, (103, 114): 4, (114, 97): 6, (97, 109): 6, (109, 109): 5, (109, 101): 11, (101, 114): 13, (114, 226): 2, (226, 128): 19, (128, 153): 8, (153, 115): 4, (115, 32): 19, (32, 73): 6, (73, 110): 2, (110, 116): 11, (116, 114): 6, (111, 100): 10, (100, 117): 2, (117, 99): 2, (99, 116): 3, (116, 105): 10, (105, 111): 5, (111, 110): 9, (110, 32): 16, (32, 116): 21, (116, 111): 8, (111, 32): 8, (32, 85): 7, (85, 110): 7, (110, 105): 8, (105, 99): 9, (99, 111): 8, (100, 101): 13, (101, 32): 33, (32, 10): 1, (10, 77): 1, (77, 97): 1, (97, 114): 14, (114, 99): 1, (99, 104): 3, (104, 32): 2, (32, 51): 2, (51, 44): 1, (44, 32): 12, (32, 50): 2, (50, 48): 1, (48, 49): 1, (49, 55): 1, (55, 32): 1, (32, 194): 2, (194, 183): 2, (183, 32): 2, (32, 67): 2, (67, 111): 2, (100, 105): 3, (105, 110): 29, (110, 103): 14, (103, 32): 9, (50, 53): 1, (53, 32): 1, (111, 109): 3, (101, 110): 6, (116, 115): 5, (115, 10): 1, (10, 10): 

In [52]:
def pprint_stats(stats):
    print(sorted(((v, k) for k, v in stats.items()), reverse=True))

In [53]:
pprint_stats(stats)

[(33, (101, 32)), (29, (105, 110)), (21, (32, 116)), (21, (32, 97)), (19, (226, 128)), (19, (115, 32)), (17, (32, 105)), (16, (110, 32)), (15, (240, 159)), (15, (116, 32)), (15, (97, 110)), (14, (116, 104)), (14, (110, 103)), (14, (97, 114)), (13, (110, 100)), (13, (101, 114)), (13, (100, 101)), (13, (100, 32)), (12, (44, 32)), (11, (110, 116)), (11, (109, 101)), (11, (32, 115)), (10, (116, 105)), (10, (116, 101)), (10, (111, 100)), (9, (111, 114)), (9, (111, 110)), (9, (105, 99)), (9, (103, 32)), (9, (32, 102)), (8, (128, 153)), (8, (116, 111)), (8, (114, 101)), (8, (114, 32)), (8, (111, 32)), (8, (110, 105)), (8, (108, 101)), (8, (104, 101)), (8, (99, 111)), (8, (32, 111)), (7, (159, 135)), (7, (159, 133)), (7, (115, 116)), (7, (114, 111)), (7, (111, 117)), (7, (104, 97)), (7, (85, 110)), (7, (32, 119)), (7, (32, 85)), (6, (239, 189)), (6, (140, 240)), (6, (128, 140)), (6, (118, 101)), (6, (117, 115)), (6, (116, 114)), (6, (115, 44)), (6, (114, 105)), (6, (114, 97)), (6, (101, 115)),

In [54]:
chr(101), chr(32)
# ... so, a lot of words end with 'e' -> makes sense
pair_to_replace = (101, 32)  # 'e' + ' '

In [55]:
# first token not present in the vocab is 256, so we will use it as a new symbol
new_symbol = 256

In [56]:
def replace_pair(tokens, pair, new_symbol):
    i = 0
    while i < len(tokens) - 1:
        if tokens[i] == pair[0] and tokens[i + 1] == pair[1]:
            tokens[i] = new_symbol
            del tokens[i + 1]
        else:
            i += 1
    return tokens

In [57]:
tokens = replace_pair(tokens, pair_to_replace, new_symbol)

In [58]:
stats = get_stats(tokens)
pprint_stats(stats)

[(29, (105, 110)), (19, (226, 128)), (19, (115, 32)), (16, (110, 32)), (16, (32, 116)), (15, (240, 159)), (15, (116, 32)), (15, (97, 110)), (15, (32, 97)), (14, (116, 104)), (14, (110, 103)), (14, (97, 114)), (14, (32, 105)), (13, (110, 100)), (13, (101, 114)), (13, (100, 32)), (12, (44, 32)), (11, (110, 116)), (10, (116, 105)), (10, (111, 100)), (9, (116, 101)), (9, (111, 114)), (9, (111, 110)), (9, (105, 99)), (9, (103, 32)), (9, (32, 102)), (8, (128, 153)), (8, (116, 111)), (8, (114, 32)), (8, (111, 32)), (8, (110, 105)), (8, (100, 101)), (8, (99, 111)), (8, (32, 115)), (7, (159, 135)), (7, (159, 133)), (7, (115, 116)), (7, (114, 111)), (7, (111, 117)), (7, (109, 101)), (7, (104, 97)), (7, (85, 110)), (7, (32, 111)), (6, (256, 97)), (6, (239, 189)), (6, (140, 240)), (6, (128, 140)), (6, (117, 115)), (6, (116, 114)), (6, (115, 44)), (6, (114, 105)), (6, (114, 97)), (6, (104, 256)), (6, (101, 115)), (6, (101, 110)), (6, (97, 116)), (6, (97, 109)), (6, (32, 119)), (6, (32, 100)), (6, (

In [59]:
top_pair = max(stats, key=stats.get)  # get the most frequent pair, elegant Pythonic way
print("Top pair:", top_pair, "->", chr(top_pair[0]), chr(top_pair[1]))

Top pair: (105, 110) -> i n


In [60]:
final_vocab_size = 276

In [61]:
def bpe(tokens, final_vocab_size):
    """Assuming we got utf-8 encoded tokens as input"""
    new_symbol = 256
    merges = {}
    while new_symbol < final_vocab_size:
        stats = get_stats(tokens)
        top_pair = max(stats, key=stats.get)
        tokens = replace_pair(tokens, top_pair, new_symbol)
        merges[top_pair] = new_symbol
        new_symbol += 1
    return tokens, merges

In [62]:
tokens, merges = bpe(tokens, final_vocab_size)

In [63]:
pprint_stats(get_stats(tokens))

[(7, (275, 271)), (7, (271, 268)), (7, (261, 266)), (7, (260, 135)), (7, (260, 133)), (7, (115, 116)), (7, (111, 117)), (7, (32, 115)), (6, (263, 32)), (6, (259, 111)), (6, (257, 140)), (6, (239, 189)), (6, (140, 260)), (6, (101, 110)), (6, (97, 109)), (5, (256, 32)), (5, (116, 114)), (5, (116, 105)), (5, (115, 267)), (5, (114, 111)), (5, (111, 102)), (5, (111, 32)), (5, (109, 109)), (5, (108, 101)), (5, (100, 101)), (5, (97, 116)), (5, (46, 32)), (4, (274, 256)), (4, (273, 258)), (4, (268, 256)), (4, (261, 32)), (4, (117, 115)), (4, (114, 97)), (4, (111, 103)), (4, (110, 111)), (4, (109, 265)), (4, (108, 108)), (4, (105, 270)), (4, (105, 108)), (4, (103, 114)), (4, (103, 111)), (4, (102, 32)), (4, (101, 262)), (4, (97, 108)), (4, (32, 275)), (4, (32, 119)), (4, (32, 97)), (3, (272, 32)), (3, (269, 104)), (3, (264, 102)), (3, (262, 116)), (3, (258, 111)), (3, (256, 115)), (3, (256, 99)), (3, (256, 97)), (3, (121, 32)), (3, (119, 104)), (3, (118, 265)), (3, (117, 264)), (3, (116, 265)),

In [64]:
pprint(merges)

{(32, 116): 259,
 (44, 32): 267,
 (85, 110): 275,
 (97, 110): 261,
 (97, 114): 262,
 (100, 32): 266,
 (101, 114): 265,
 (105, 99): 271,
 (105, 110): 256,
 (111, 100): 268,
 (111, 110): 270,
 (111, 114): 272,
 (115, 32): 258,
 (116, 32): 264,
 (226, 128): 257,
 (240, 159): 260,
 (256, 103): 263,
 (256, 116): 269,
 (257, 153): 273,
 (259, 104): 274}


In [65]:
def merge(ids, pair, idx):
    # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
    newids = []
    i = 0
    while i < len(ids):
        # if we are not at the very last position AND the pair matches, replace it
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [66]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

In [67]:
def decode(ids):
    # given idx (list of integers), return Python string
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

In [68]:
def encode(text):
    # given Python string, return list of integers (the tokens))
    tokens = text.encode("utf-8")
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break  # no more merges possible
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

In [69]:
print(decode(encode("hello world")))

hello world


### Speficic tokenizers for well-known LLMs

In [70]:
# allows us to block certain pairs from being merged (GPT-2)
pat = re.compile(
    r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
)

In [71]:
test_str = "Hello, I'm testing the tokenizer. It's working, isn't it great?"
print(re.findall(pat, test_str))

['Hello', ',', ' I', "'m", ' testing', ' the', ' tokenizer', '.', ' It', "'s", ' working', ',', ' isn', "'t", ' it', ' great', '?']


In [72]:
enc = tiktoken.get_encoding("gpt2")
print(enc.encode(test_str))

[15496, 11, 314, 1101, 4856, 262, 11241, 7509, 13, 632, 338, 1762, 11, 2125, 470, 340, 1049, 30]


In [73]:
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode(test_str))

[9906, 11, 358, 2846, 7649, 279, 47058, 13, 1102, 596, 3318, 11, 4536, 956, 433, 2294, 30]


### Special tokens

In [74]:
# for example '<|endoftext|>' ...

### Different modalities

In [75]:
# visual patches, like in SORA